In [16]:
import graphviz
from datetime import datetime
import json
import numpy as np
import time

In [17]:
names = ['smallRandom','xsmallComplex','smallComplex','mediumRandom','MediumComplex','largeComplex','xlargeComplex','xxlargeComplex']

In [18]:
with open('./graphs/smallRandom.json', 'r') as f:
    data = json.load(f)

nodes = data['nodes']

In [19]:
def create_successors():
    for node in nodes:
        nodes[node]['Successors']=[]
    for node in nodes :
        depend = nodes[node]['Dependencies']
        if len(depend)>0:
            for i in depend :
                nodes[str(i)]['Successors'].append(node)


In [20]:
# create_successors()
# nodes

In [21]:
def weight(node):
    pt = node['Data'].split(':')
    time_s = float(pt[0])*3600 + float(pt[1])*60 + float(pt[2])
    return time_s

In [22]:
def add_ranks():
    for node in nodes:
        nodes[node]['rank'] = weight(nodes[node])
        nodes[node]['rank_calculated'] = False

# for node in nodes.values() :
#     print(node['rank'])
#     print(node['rank_calculated'])

In [23]:
def calculate_rank(node):
    if not nodes[node]['rank_calculated']:
        succ = nodes[node]['Successors']     
        rank = weight(nodes[node])
        if len(succ) > 0:
            for i in succ:
                if not nodes[str(i)]['rank_calculated']:
                    # print("couldn't calculate rank")
                    return 1  
            weights = [nodes[str(i)]['rank'] for i in succ]
            # print('node weight:',rank)
            # print('successors:',succ)
            # print('succ weights:',weights)
            max_weights = max(weights)
            # print('max:',max_weights)
            rank += max_weights
        nodes[node]['rank'] = rank
        # print('calculated rank:', rank)
        nodes[node]['rank_calculated'] = True
    return 0

In [24]:
def graph_update():
    count = 0
    for node in reversed(nodes.keys()):
        # print('node:',node)
        count += calculate_rank(node)
    print('not calculated ranks:',count)    
    return count

In [25]:
def calculate_ranks():
    epoch = 1
    count = 1
    while count > 0:
        print('epoch:',epoch)
        count = graph_update()
        epoch+=1
    print('rank calculation done')

In [26]:
def make_priority_list():
    node_list = []
    rank_list = []
    for node in nodes:
        node_list.append(node)
        rank_list.append(nodes[node]['rank'])

    # print(node_list)
    # print(rank_list)
    # indexes = np.flip(np.argsort(rank_list))
    # sorted_nodes = [node_list[i] for i in indexes]
    sorted_nodes = [x for _,x in sorted(zip(rank_list,node_list), reverse=True)]
    return sorted_nodes

In [27]:
def process_graph():
    create_successors()
    add_ranks()
    calculate_ranks()
    priority_list = make_priority_list()
    return priority_list
    

In [28]:
start_time = time.time()
priority_list = process_graph()
duration = time.time() - start_time
# print(priority_list)
# print(len(nodes))
# print(len(priority_list))
print('duration:', duration)


epoch: 1
not calculated ranks: 0
rank calculation done
duration: 0.0


In [31]:
def calculate_execution_time(priority_list,n_cores):
    execution_times = [0]*n_cores
    for node in priority_list :
        task_time = weight(nodes[node])
        i = np.argmin(execution_times)
        execution_times[i] += task_time
    return execution_times

In [32]:
calculate_execution_time(priority_list,2)

[19641.855386000003, 19558.9887496]